In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import decomposition
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import MinMaxScaler
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import euclidean_distances
import random
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from random import shuffle
import time

In [0]:
pip install git+https://github.com/samueljackson92/coranking.git


  Cloning https://github.com/samueljackson92/coranking.git to /tmp/pip-req-build-7bd9yw48
  Running command git clone -q https://github.com/samueljackson92/coranking.git /tmp/pip-req-build-7bd9yw48
  Created wheel for coranking: filename=coranking-0.1.1-cp36-cp36m-linux_x86_64.whl size=87035 sha256=b5673c9f1116dcfbdaf595d52e4472643bacedc747e33c7ca5aed28e48273561
  Stored in directory: /tmp/pip-ephem-wheel-cache-jhohznqt/wheels/8e/e0/11/4ae911b6fc472b8bf6087e29d861d2b798c119603adf4b93fa
Successfully built coranking


In [0]:
import coranking
from coranking.metrics import trustworthiness, continuity, LCMC


In [0]:
link1='https://drive.google.com/open?id=1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW'

fluff, id = link1.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('train.csv') 
train_df  = pd.read_csv('train.csv')

1K2_KnRWrKl0IfCUmKLkFerjITQt8B4lW


In [0]:
concat_df =train_df.copy()

#columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice']
columns_to_drop = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageCars','Id','SalePrice','MasVnrArea', 'GarageYrBlt', 'LotFrontage']
concat_df.drop(columns_to_drop, axis=1, inplace=True)

nan_columns=concat_df.columns[concat_df.isna().any()].tolist()
numerical_cols = [f for f in concat_df.columns if concat_df.dtypes[f] != 'object']
containNumNans = list(set(nan_columns) & set(numerical_cols))

scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(concat_df[numerical_cols])
concat_df.loc[:,numerical_cols] = scaled_values

concat_df = pd.get_dummies(concat_df)

In [0]:
labels = train_df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(concat_df, labels, test_size=0.25, random_state=42)

GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
GB_model = GBest.fit(X_train, y_train)
preds = GBest.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))


RMSE: 24475.743133


###UMAP

In [0]:
import umap
from prettytable import PrettyTable
t_mds_10k = PrettyTable(['RMSE_onUMAPData', 'Dimensions', 'UMAP_time','GBR_time' ,'total_time', 'trust_UMAP','cont_UMAP' ,'lcmc_UMAP'])


def MDS_Decision_Tree (training_data, components,size):
  #Reducing to mentioned components
  #print("starting mds")
  mds_start_time = time.time()
  reducer = umap.UMAP(random_state=42,n_components=components)
  training_data_mds = reducer.fit_transform(training_data[:size])
  mds_time = time.time() - mds_start_time
  #print("ending mds")
  
  #Applying Decision Tree
  X_train, X_test, y_train, y_test = train_test_split(training_data_mds, labels[:size], test_size=0.25, random_state=49)
  clf_start_time = time.time()

  GBest = ensemble.GradientBoostingRegressor(n_estimators=7000, learning_rate=0.01, max_depth=10, max_features='sqrt',min_samples_leaf=5, min_samples_split=15, loss='ls')
  GB_model = GBest.fit(X_train, y_train)
  preds = GBest.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, preds))


  clf_time = time.time() - clf_start_time
  score=rmse
  score = str(round(score, 4))

  total_time = mds_time+clf_time
  mds_time = str(round(mds_time, 4))
  clf_time = str(round(clf_time, 4))
  total_time = str(round(total_time, 4))


  trust_mds = 0
  cont_mds = 0
  lcmc_mds = 0
  
  Q = coranking.coranking_matrix(training_data[:size], training_data_mds)
  trust_mds = trustworthiness(Q, min_k=1, max_k=50).sum()/50
  cont_mds = continuity(Q, min_k=1, max_k=50).sum()/50
  lcmc_mds = LCMC(Q, min_k=1, max_k=50).sum()/50

  trust_mds = str(round(trust_mds, 4))
  cont_mds = str(round(cont_mds, 4))
  lcmc_mds = str(round(lcmc_mds, 4))
  
  #print(components)
  print("{}, {}, {}, {}, {}, {}, {}, {} ".format( score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds))
  print("*"*40)
  t_mds_10k.add_row([score, components, mds_time, clf_time ,total_time, trust_mds,cont_mds ,lcmc_mds])
  return

In [0]:
dimensions=266
size=1460
while dimensions >= 2:
  MDS_Decision_Tree (concat_df, dimensions,size)
  dimensions=int(dimensions/2)

38351.2949, 266, 64.3082, 37.5567, 101.865, 0.9082, 0.9167, 0.3717 
****************************************
39960.0704, 133, 31.4243, 25.8047, 57.2289, 0.9083, 0.9162, 0.3735 
****************************************
38256.1087, 66, 16.7027, 19.7665, 36.4692, 0.9087, 0.9165, 0.3727 
****************************************
38630.5853, 33, 8.2214, 13.2828, 21.5041, 0.9091, 0.9169, 0.374 
****************************************
39254.6354, 16, 5.6077, 11.3242, 16.9319, 0.9085, 0.9167, 0.3743 
****************************************
40215.0952, 8, 4.891, 7.0121, 11.9031, 0.9085, 0.9161, 0.3731 
****************************************
41849.5026, 4, 4.6057, 6.6915, 11.2972, 0.903, 0.9153, 0.3619 
****************************************
50489.1941, 2, 4.1748, 4.8096, 8.9844, 0.8803, 0.9076, 0.3077 
****************************************


In [0]:
print(t_mds_10k)

+-----------------+------------+-----------+----------+------------+------------+-----------+-----------+
| RMSE_onUMAPData | Dimensions | UMAP_time | GBR_time | total_time | trust_UMAP | cont_UMAP | lcmc_UMAP |
+-----------------+------------+-----------+----------+------------+------------+-----------+-----------+
|    38351.2949   |    266     |  64.3082  | 37.5567  |  101.865   |   0.9082   |   0.9167  |   0.3717  |
|    39960.0704   |    133     |  31.4243  | 25.8047  |  57.2289   |   0.9083   |   0.9162  |   0.3735  |
|    38256.1087   |     66     |  16.7027  | 19.7665  |  36.4692   |   0.9087   |   0.9165  |   0.3727  |
|    38630.5853   |     33     |   8.2214  | 13.2828  |  21.5041   |   0.9091   |   0.9169  |   0.374   |
|    39254.6354   |     16     |   5.6077  | 11.3242  |  16.9319   |   0.9085   |   0.9167  |   0.3743  |
|    40215.0952   |     8      |   4.891   |  7.0121  |  11.9031   |   0.9085   |   0.9161  |   0.3731  |
|    41849.5026   |     4      |   4.6057  |  